In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pyodbc
import cx_Oracle

from lifelines.plotting import plot_lifetimes      # Lifeline package for the Survival Analysis
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
cust_data = pd.read_csv("D:\TBL_TRN_CUSTOMER_DETAILS.csv")
coll_data = pd.read_csv("D:\TBL_TRN_COLLECTION_DETAILS.csv")

C:\Users\5029874\AppData\Local\Continuum\anaconda3\envs\trainingenv\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (9,10,11,12,15,39,50,61,71,95,106,107,110,125,138,147,148,151,156,163,164,166,167,168,184,185,186,187,188,205,207,210,211,212,213,214,215,217,218,219,220,221,223) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\5029874\AppData\Local\Continuum\anaconda3\envs\trainingenv\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (42,43,48,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import gc

In [20]:
gc.collect()

0

In [5]:
cust_data = cust_data[['AGMTNO', 'AGMT_DATE', 'PORTFOLIO', 'CHANNEL', 'PAYMENT_TYPE',
       'AREA_CODE', 'DEALER_TYPE', 'DEALER_CODE', 'MAINDEALER_CODE',
       'EXECUTIVE_CODE', 'PRODUCT_CODE', 'MODEL_CODE', 'SCHEME_TYPE',
       'SCHEME_CODE', 'ADVANCE_EMI_COUNT', 'TENUR', 'ROI', 'EMI',
       'PORCESSING_FEE', 'STAMPING_CHARGES', 'ASSET_COST', 'LOAN_AMOUNT',
       'DOWN_PAYMENT', 'INITITAL_HIRE', 'LTV', 'AMORT_IRR', 'CUSTOMER_IRR',
       'NET_IRR', 'VECHICAL_INSURANCE_AMT', 'SPECIAL_INSURANCE',
       'SPECIAL_INSURANCE_FOR', 'SPECIAL_INSURANCE_PRODUCT',
       'SPECIAL_INSURANCE_AMOUNT', 'GENDER', 'PANNO', 'DRIVING_LICENSE',
       'PASSPORT', 'AADHAR', 'RATION', 'EXISTING_CUSTOMER', 'EMPLOYMENT_TYPE',
       'CUSTOMER_PROFILE_CODE', 'RESIDENT_TYPE', 'BANK_ACCOUNT', 'PINCODE',
       'CITY', 'STATE', 'TRN_MONTH', 'TRN_YEAR', 'FIN_YEAR',
       'ACTUAL_LOAN_AMOUNT', 'PRODUCT_CUST_IRR',
       'PRODUCT_AMORT_IRR', 'PRODUCT_TENUR', 'PF_SERVICE_TAX',
       'CIBIL_SCORE', 'COBORROWER', 'GUARANTOR', 'RELIGION',
       'CIBIL_HIT', 'CIBIL_OVERDUE', 'CIBIL_SCORE_COBORROWER',
       'CIBIL_SCORE_GURANTOR', 'LAST_EMI_DATE','FIRST_EMI_DATE', 'STATUS', 'ADDRESS_TYPE',
       'ADDRESS_SLNO', 'SUB_STATUS', 'DOB','APP_SCORE_DESC']]

In [6]:
#cust_data.info(max_cols=300)
cust_data.head()

,AGMTNO,AGMT_DATE,PORTFOLIO,CHANNEL,PAYMENT_TYPE,AREA_CODE,DEALER_TYPE,DEALER_CODE,MAINDEALER_CODE,EXECUTIVE_CODE,...,CIBIL_SCORE_COBORROWER,CIBIL_SCORE_GURANTOR,LAST_EMI_DATE,FIRST_EMI_DATE,STATUS,ADDRESS_TYPE,ADDRESS_SLNO,SUB_STATUS,DOB,APP_SCORE_DESC
0,TN3000TW0039449,2016-01-02,TW,ECF,ECS,3000,DEALER,1001,1001,5008949,...,NaN,0.0,2016-11-07,2016-02-07,M,RS,1,NaN,1990-03-01,LOW RISK
1,GJ3031TW10471,2015-06-19,TW,ECF,ELC,3031,DEALER,1774,1774,5009503,...,NaN,0.0,2016-07-07,2015-08-07,M,RS,1,NaN,1978-06-12,LOW RISK
2,GJ3031TW0012554,2015-12-04,TW,ECF,ELC,3031,DEALER,3008,3008,5012511,...,NaN,0.0,2016-12-07,2016-01-07,M,RS,1,NaN,1977-05-09,LOW RISK
3,GJ3031TW10521,2015-06-23,TW,ECF,ELC,3031,DEALER,1774,1774,5009503,...,NaN,0.0,2016-07-07,2015-08-07,M,RO,1,NaN,1978-12-31,MEDIUM RISK
4,TN3000TW0039724,2016-01-11,TW,ECF,ELC,3000,DEALER,1001,1001,5008949,...,NaN,0.0,2018-07-07,2016-02-07,M,RS,1,NaN,1993-06-13,MEDIUM RISK


In [7]:
cust_data['AGMT_DATE'] = cust_data['AGMT_DATE'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))

In [8]:
def Mark_DOB(row):
    if isinstance(row['DOB'], str):
        return datetime.datetime.strptime(row['DOB'], "%Y-%m-%d")
    else:
        return row['AGMT_DATE']

In [9]:
cust_data['DOB'] = cust_data.apply(Mark_DOB, axis=1)

In [10]:
from dateutil.relativedelta import relativedelta

In [11]:
def get_age(row):
    return relativedelta(row['AGMT_DATE'],  row['DOB']).years

In [12]:
cust_data['AGE'] = cust_data.apply(get_age, axis=1)

In [13]:
cust_data = cust_data[cust_data.FIRST_EMI_DATE.notnull()]
cust_data = cust_data[cust_data.LAST_EMI_DATE.notnull()]

In [14]:
cust_data['LAST_EMI_DATE'] = cust_data['LAST_EMI_DATE'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
cust_data['FIRST_EMI_DATE'] = cust_data['FIRST_EMI_DATE'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))

In [15]:
def get_duration(row):
    return relativedelta(row['LAST_EMI_DATE'], row['FIRST_EMI_DATE']).months

In [16]:
cust_data['LOAN_DURATION'] = cust_data.apply(get_duration, axis=1)

In [17]:
cust_data = cust_data[['AGMTNO', 'AGMT_DATE', 'PORTFOLIO', 'CHANNEL', 'PAYMENT_TYPE',
       'AREA_CODE', 'DEALER_TYPE', 'DEALER_CODE', 'MAINDEALER_CODE',
       'EXECUTIVE_CODE', 'PRODUCT_CODE', 'MODEL_CODE', 'SCHEME_TYPE',
       'SCHEME_CODE', 'ADVANCE_EMI_COUNT', 'TENUR', 'ROI', 'EMI',
       'PORCESSING_FEE', 'STAMPING_CHARGES', 'ASSET_COST', 'LOAN_AMOUNT',
       'DOWN_PAYMENT', 'INITITAL_HIRE', 'LTV', 'AMORT_IRR', 'CUSTOMER_IRR',
       'NET_IRR', 'VECHICAL_INSURANCE_AMT', 'SPECIAL_INSURANCE',
       'SPECIAL_INSURANCE_FOR', 'SPECIAL_INSURANCE_PRODUCT',
       'SPECIAL_INSURANCE_AMOUNT', 'GENDER', 'PANNO', 'DRIVING_LICENSE',
       'PASSPORT', 'AADHAR', 'RATION', 'EXISTING_CUSTOMER', 'EMPLOYMENT_TYPE',
       'CUSTOMER_PROFILE_CODE', 'RESIDENT_TYPE', 'BANK_ACCOUNT', 'PINCODE',
       'CITY', 'STATE', 'TRN_MONTH', 'TRN_YEAR', 'FIN_YEAR',
       'ACTUAL_LOAN_AMOUNT', 'PRODUCT_CUST_IRR',
       'PRODUCT_AMORT_IRR', 'PRODUCT_TENUR', 'PF_SERVICE_TAX',
       'CIBIL_SCORE', 'COBORROWER', 'GUARANTOR', 'RELIGION',
       'CIBIL_HIT', 'CIBIL_OVERDUE', 'CIBIL_SCORE_COBORROWER',
       'CIBIL_SCORE_GURANTOR', 'LOAN_DURATION', 'STATUS', 'ADDRESS_TYPE',
       'ADDRESS_SLNO', 'SUB_STATUS', 'AGE', 'APP_SCORE_DESC']]

In [18]:
cust_data = cust_data[cust_data.CHANNEL.notnull()]
cust_data = cust_data[cust_data.DOWN_PAYMENT.notnull()]
cust_data = cust_data[cust_data.ROI.notnull()]
cust_data = cust_data[cust_data.GENDER.notnull()]
cust_data = cust_data[cust_data.EMPLOYMENT_TYPE.notnull()]
cust_data = cust_data[cust_data.CUSTOMER_PROFILE_CODE.notnull()]
cust_data = cust_data[cust_data.CITY.notnull()]
cust_data = cust_data[cust_data.RELIGION.notnull()]

In [19]:
cust_data['VECHICAL_INSURANCE_AMT'].fillna(0, inplace=True)
cust_data['SPECIAL_INSURANCE_FOR'].fillna('N/A', inplace=True)
cust_data['SPECIAL_INSURANCE_PRODUCT'].fillna('N/A', inplace=True)
cust_data['SPECIAL_INSURANCE_AMOUNT'].fillna(0, inplace=True)
cust_data['EXISTING_CUSTOMER'].fillna('N/A', inplace=True)
cust_data['RESIDENT_TYPE'].fillna('N/A', inplace=True)
cust_data['CIBIL_SCORE'].fillna(0, inplace=True)
cust_data['CIBIL_HIT'].fillna('N/A', inplace=True)
cust_data['CIBIL_SCORE_COBORROWER'].fillna(0, inplace=True)
cust_data['CIBIL_SCORE_GURANTOR'].fillna(0, inplace=True)
cust_data['SUB_STATUS'].fillna('L', inplace=True)

In [21]:
cust_data = cust_data.dropna()

In [37]:
def age_bin(row):
    if row['AGE'] <= 25:
        return 1
    elif row['AGE'] > 25 & row['AGE'] <= 35:
        return 2
    elif row['AGE'] > 35 & row['AGE'] <= 45:
        return 3
    elif row['AGE'] > 45 & row['AGE'] <= 55:
        return 4
    else:
        return 5

In [38]:
cust_data['AGE_BIN'] = cust_data.apply(age_bin, axis=1)

In [36]:
coll_data = coll_data[['AGMTNO', 'MOB', 'STATUS']]

In [23]:
coll_data = coll_data[coll_data['AGMTNO'].str.contains("TW")]

In [24]:
coll_data = coll_data.groupby(['AGMTNO'], sort=False)['MOB'].max()

In [25]:
coll_data = pd.DataFrame({'AGMTNO':coll_data.index, 'MOB':coll_data.values})

In [26]:
Final_data = pd.merge(cust_data, coll_data, on='AGMTNO', how='left')

In [27]:
Final_data["MOB"] = Final_data["MOB"].fillna(0.0).astype(int)

In [53]:
Final_data.head()

,AGMTNO,AGMT_DATE,PORTFOLIO,CHANNEL,PAYMENT_TYPE,AREA_CODE,DEALER_TYPE,DEALER_CODE,MAINDEALER_CODE,EXECUTIVE_CODE,...,CIBIL_SCORE_GURANTOR,LOAN_DURATION,STATUS,ADDRESS_TYPE,ADDRESS_SLNO,SUB_STATUS,AGE,APP_SCORE_DESC,MOB,Target
0,TN3000TW0039449,2016-01-02,TW,ECF,ECS,3000,DEALER,1001,1001,5008949,...,0.0,9,M,RS,1,L,25,LOW RISK,0,0
1,GJ3031TW10471,2015-06-19,TW,ECF,ELC,3031,DEALER,1774,1774,5009503,...,0.0,11,M,RS,1,L,37,LOW RISK,0,0
2,GJ3031TW0012554,2015-12-04,TW,ECF,ELC,3031,DEALER,3008,3008,5012511,...,0.0,11,M,RS,1,L,38,LOW RISK,0,0
3,GJ3031TW10521,2015-06-23,TW,ECF,ELC,3031,DEALER,1774,1774,5009503,...,0.0,11,M,RO,1,L,36,MEDIUM RISK,0,0
4,TN3000TW0039724,2016-01-11,TW,ECF,ELC,3000,DEALER,1001,1001,5008949,...,0.0,5,M,RS,1,L,22,MEDIUM RISK,0,0


In [30]:
def mark_target(row):
    if row['MOB'] == 0:
        return 0
    else:
        return 1

In [31]:
Final_data['Target'] = Final_data.apply(mark_target, axis=1)

In [52]:
Final_data[['APP_SCORE_DESC','Target']].loc[(Final_data['Target'] == 1)].groupby(['APP_SCORE_DESC']).count()

,Target
APP_SCORE_DESC,
HIGH RISK,26671
LOW RISK,134321
MEDIUM RISK,179795


In [33]:
Final_data.to_csv("D:\cust_coll_data.csv")

In [42]:
Final_data.AGMT_DATE.min()
# 2010> 2345717

Timestamp('2014-08-01 00:00:00')

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
from scipy import spatial

In [57]:
#cosine_similarity([1, 0, -1], [-1,-1, 0])
1 - spatial.distance.cosine(X_Norm_avg, [-1,-1, 0])

-0.5

In [62]:
Final_train = Final_data.loc[Final_data['AGMT_DATE'] < datetime.datetime.strptime("2018-01-01", "%Y-%m-%d")]

In [66]:
Final_train = Final_train.drop(['APP_SCORE_DESC'], axis=1)

In [75]:
Final_train = Final_train.drop(['AGMT_DATE'], axis=1)

In [78]:
Final_train = Final_train.drop(['AGMTNO'], axis=1)

In [82]:
Final_train = Final_train.drop(['PORTFOLIO'], axis=1)

In [68]:
Final_test = Final_data.loc[Final_data['AGMT_DATE'] >= datetime.datetime.strptime("2018-01-01", "%Y-%m-%d")]

In [71]:
Final_test = Final_test.drop(['APP_SCORE_DESC'], axis=1)

In [76]:
Final_test = Final_test.drop(['AGMT_DATE'], axis=1)

In [158]:
Final_test = Final_test.drop(['AGMTNO'], axis=1)

In [ ]:
Final_test = Final_test.drop(['PORTFOLIO'], axis=1)

In [147]:
Final_train[['cluster_no', 'Target']].groupby(['cluster_no'])['Target'].agg(['sum','count']) 

,sum,count
cluster_no,,
0,29844,140039
1,104693,523464
2,61998,263859
3,95637,481572
4,31209,248167


In [180]:
Final_test#.loc[Final_train['cluster_no'] == 1]

,AGMTNO,CHANNEL,PAYMENT_TYPE,AREA_CODE,DEALER_TYPE,DEALER_CODE,MAINDEALER_CODE,EXECUTIVE_CODE,PRODUCT_CODE,MODEL_CODE,...,CIBIL_SCORE_GURANTOR,LOAN_DURATION,STATUS,ADDRESS_TYPE,ADDRESS_SLNO,SUB_STATUS,AGE,MOB,Target,cluster_no
35086,PB3089TW0002724,0,0,3089,0,0,0,0,0,0,...,0.0,5,0,0,1,0,33,0,0,4
35151,MH3028TW0041575,0,1,3028,1,1,1,1,0,1,...,0.0,5,0,1,1,0,29,0,0,2
53806,PB3050TW0009195,0,0,3050,0,2,2,2,1,2,...,0.0,11,0,1,1,0,50,0,0,0
70445,UP3033TW0022251,0,0,3033,1,3,3,3,0,3,...,0.0,11,1,0,1,0,41,0,0,4
75697,AP3042TW0073412,0,1,3042,1,4,4,4,1,2,...,0.0,11,0,0,1,0,33,0,0,3
92237,MH3061TW0016319,0,1,3061,0,5,5,5,1,4,...,0.0,11,0,0,1,0,25,0,0,0
101451,AP3019TW0074854,0,1,3019,0,6,6,6,2,5,...,0.0,5,0,0,1,0,30,0,0,1
101452,MH3061TW0016838,0,1,3061,0,7,7,7,0,6,...,0.0,11,0,1,1,0,31,0,0,2
112873,UP3033TW0023517,0,0,3033,1,8,8,8,0,0,...,0.0,2,0,1,1,0,66,0,0,4
119430,JH3053TW0036101,0,0,3053,1,9,9,9,1,2,...,0.0,11,2,1,1,1,26,12,1,3


In [72]:
from pandas.api.types import is_string_dtype

In [73]:
for i in Final_train.columns:
    if is_string_dtype(Final_train[i]):
        Final_train[i] = pd.factorize(Final_train[i])[0]

In [160]:
for i in Final_test.columns:
    if is_string_dtype(Final_test[i]):
        Final_test[i] = pd.factorize(Final_test[i])[0]

In [187]:
for i in Final_train.columns:
    #if data[i].isnull().sum() != 0:
    if Final_train[i].unique().size < 15:
        print("column {} has ---unique----> {} ---null----> {} ".format(i,Final_train[i].unique().size, Final_train[i].isnull().sum()))

column CHANNEL has ---unique----> 2 ---null----> 0 
column PAYMENT_TYPE has ---unique----> 8 ---null----> 0 
column DEALER_TYPE has ---unique----> 2 ---null----> 0 
column PRODUCT_CODE has ---unique----> 3 ---null----> 0 
column SCHEME_TYPE has ---unique----> 3 ---null----> 0 
column ADVANCE_EMI_COUNT has ---unique----> 8 ---null----> 0 
column SPECIAL_INSURANCE has ---unique----> 2 ---null----> 0 
column SPECIAL_INSURANCE_FOR has ---unique----> 3 ---null----> 0 
column SPECIAL_INSURANCE_PRODUCT has ---unique----> 3 ---null----> 0 
column SPECIAL_INSURANCE_AMOUNT has ---unique----> 5 ---null----> 0 
column GENDER has ---unique----> 2 ---null----> 0 
column PANNO has ---unique----> 2 ---null----> 0 
column DRIVING_LICENSE has ---unique----> 2 ---null----> 0 
column PASSPORT has ---unique----> 2 ---null----> 0 
column AADHAR has ---unique----> 2 ---null----> 0 
column RATION has ---unique----> 2 ---null----> 0 
column EXISTING_CUSTOMER has ---unique----> 3 ---null----> 0 
column EMPLOYME

In [ ]:
Final_train.loc[(Final_Train['Target'] == 1)]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [94]:
X_Norm = Final_train.loc[(Final_train['Target'] == 0)].iloc[:, 0:67]

In [96]:
X_Norm.head()

,CHANNEL,PAYMENT_TYPE,AREA_CODE,DEALER_TYPE,DEALER_CODE,MAINDEALER_CODE,EXECUTIVE_CODE,PRODUCT_CODE,MODEL_CODE,SCHEME_TYPE,...,CIBIL_OVERDUE,CIBIL_SCORE_COBORROWER,CIBIL_SCORE_GURANTOR,LOAN_DURATION,STATUS,ADDRESS_TYPE,ADDRESS_SLNO,SUB_STATUS,AGE,MOB
0,0,0,3000,0,0,0,0,0,0,0,...,0.0,0.0,0.0,9,0,0,1,0,25,0
1,0,1,3031,0,1,1,1,0,1,0,...,0.0,0.0,0.0,11,0,0,1,0,37,0
2,0,1,3031,0,2,2,2,0,1,1,...,0.0,0.0,0.0,11,0,0,1,0,38,0
3,0,1,3031,0,1,1,1,1,2,0,...,0.0,0.0,0.0,11,0,1,1,0,36,0
4,0,1,3000,0,0,0,0,1,3,1,...,0.0,0.0,0.0,5,0,0,1,0,22,0


In [97]:
Y_Norm = Final_train.loc[(Final_train['Target'] == 1)].iloc[:, 0:67]

In [98]:
Y_Norm.head()

,CHANNEL,PAYMENT_TYPE,AREA_CODE,DEALER_TYPE,DEALER_CODE,MAINDEALER_CODE,EXECUTIVE_CODE,PRODUCT_CODE,MODEL_CODE,SCHEME_TYPE,...,CIBIL_OVERDUE,CIBIL_SCORE_COBORROWER,CIBIL_SCORE_GURANTOR,LOAN_DURATION,STATUS,ADDRESS_TYPE,ADDRESS_SLNO,SUB_STATUS,AGE,MOB
7,0,2,3009,0,4,4,5,2,5,2,...,0.0,0.0,0.0,6,1,1,1,1,32,14
16,0,1,3006,0,11,3,12,0,8,2,...,0.0,0.0,0.0,5,1,1,1,1,56,11
21,0,1,3006,0,11,3,15,0,0,2,...,0.0,0.0,0.0,11,1,1,1,1,44,6
22,0,1,3007,0,14,12,16,2,4,2,...,0.0,0.0,0.0,5,1,1,1,2,29,13
25,0,2,3007,0,14,12,16,2,9,2,...,0.0,0.0,0.0,11,1,0,1,1,28,10


In [100]:
X_Norm_avg = []
for i in X_Norm.columns:
    X_Norm_avg.append(X_Norm[i].unique().mean())

In [101]:
Y_Norm_avg = []
for i in Y_Norm.columns:
    Y_Norm_avg.append(Y_Norm[i].unique().mean())

In [111]:
1-spatial.distance.cosine([2,3,5,6], [0.2,56,0.8,0.13])

0.35945588485439606

In [123]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

In [124]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [149]:
try:
    for k in range (1, 11):
        # Create a kmeans model on our data, using k clusters.  random_state helps ensure that the algorithm returns the same results each time.
        kmeans_model = KMeans(n_clusters=k, random_state=1).fit(Final_train.iloc[:, :])

        # These are our fitted labels for clusters -- the first cluster has label 0, and the second has label 1.
        labels = kmeans_model.labels_
 
        # Sum of distances of samples to their closest cluster center
        interia = kmeans_model.inertia_
        print("k:,{},  cost: {}".format(k,interia))
except:
    print("Process Stopped")

Process Stopped


In [130]:
model3=KMeans(n_clusters=5)

In [131]:
model3.fit(Final_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [132]:
clusassign=model3.predict(Final_train)

In [134]:
type(clusassign)

numpy.ndarray

In [135]:
Final_train['cluster_no'] =  clusassign

In [162]:
clusassign_test=model3.predict(Final_test)

In [164]:
Final_test['cluster_no'] =  clusassign_test

In [151]:
Finaltrain_AGMTNO = Final_data.loc[Final_data['AGMT_DATE'] < datetime.datetime.strptime("2018-01-01", "%Y-%m-%d")]['AGMTNO']

In [156]:
type(Finaltrain_AGMTNO.values)

numpy.ndarray

In [167]:
Final_train['AGMTNO'] = Finaltrain_AGMTNO.values

In [168]:
Finaltest_AGMTNO = Final_data.loc[Final_data['AGMT_DATE'] >= datetime.datetime.strptime("2018-01-01", "%Y-%m-%d")]['AGMTNO']

In [170]:
Final_test['AGMTNO'] = Finaltest_AGMTNO.values

In [188]:
Final_train = Final_train[['AGMTNO','CHANNEL', 'PAYMENT_TYPE', 'AREA_CODE', 'DEALER_TYPE', 'DEALER_CODE',
       'MAINDEALER_CODE', 'EXECUTIVE_CODE', 'PRODUCT_CODE', 'MODEL_CODE',
       'SCHEME_TYPE', 'SCHEME_CODE', 'ADVANCE_EMI_COUNT', 'TENUR', 'ROI',
       'EMI', 'PORCESSING_FEE', 'STAMPING_CHARGES', 'ASSET_COST',
       'LOAN_AMOUNT', 'DOWN_PAYMENT', 'INITITAL_HIRE', 'LTV', 'AMORT_IRR',
       'CUSTOMER_IRR', 'NET_IRR', 'VECHICAL_INSURANCE_AMT',
       'SPECIAL_INSURANCE', 'SPECIAL_INSURANCE_FOR',
       'SPECIAL_INSURANCE_PRODUCT', 'SPECIAL_INSURANCE_AMOUNT', 'GENDER',
       'PANNO', 'DRIVING_LICENSE', 'PASSPORT', 'AADHAR', 'RATION',
       'EXISTING_CUSTOMER', 'EMPLOYMENT_TYPE', 'CUSTOMER_PROFILE_CODE',
       'RESIDENT_TYPE', 'BANK_ACCOUNT', 'PINCODE', 'CITY', 'STATE',
       'ACTUAL_LOAN_AMOUNT',
       'PRODUCT_CUST_IRR', 'PRODUCT_AMORT_IRR', 'PRODUCT_TENUR',
       'PF_SERVICE_TAX', 'CIBIL_SCORE', 'COBORROWER', 'GUARANTOR', 'RELIGION',
       'CIBIL_HIT', 'CIBIL_OVERDUE', 'CIBIL_SCORE_COBORROWER',
       'CIBIL_SCORE_GURANTOR', 'LOAN_DURATION', 'STATUS', 'ADDRESS_TYPE',
       'ADDRESS_SLNO', 'SUB_STATUS', 'AGE', 'MOB', 'Target', 'cluster_no']]

In [189]:
Final_test = Final_test[['AGMTNO','CHANNEL', 'PAYMENT_TYPE', 'AREA_CODE', 'DEALER_TYPE', 'DEALER_CODE',
       'MAINDEALER_CODE', 'EXECUTIVE_CODE', 'PRODUCT_CODE', 'MODEL_CODE',
       'SCHEME_TYPE', 'SCHEME_CODE', 'ADVANCE_EMI_COUNT', 'TENUR', 'ROI',
       'EMI', 'PORCESSING_FEE', 'STAMPING_CHARGES', 'ASSET_COST',
       'LOAN_AMOUNT', 'DOWN_PAYMENT', 'INITITAL_HIRE', 'LTV', 'AMORT_IRR',
       'CUSTOMER_IRR', 'NET_IRR', 'VECHICAL_INSURANCE_AMT',
       'SPECIAL_INSURANCE', 'SPECIAL_INSURANCE_FOR',
       'SPECIAL_INSURANCE_PRODUCT', 'SPECIAL_INSURANCE_AMOUNT', 'GENDER',
       'PANNO', 'DRIVING_LICENSE', 'PASSPORT', 'AADHAR', 'RATION',
       'EXISTING_CUSTOMER', 'EMPLOYMENT_TYPE', 'CUSTOMER_PROFILE_CODE',
       'RESIDENT_TYPE', 'BANK_ACCOUNT', 'PINCODE', 'CITY', 'STATE',
       'ACTUAL_LOAN_AMOUNT',
       'PRODUCT_CUST_IRR', 'PRODUCT_AMORT_IRR', 'PRODUCT_TENUR',
       'PF_SERVICE_TAX', 'CIBIL_SCORE', 'COBORROWER', 'GUARANTOR', 'RELIGION',
       'CIBIL_HIT', 'CIBIL_OVERDUE', 'CIBIL_SCORE_COBORROWER',
       'CIBIL_SCORE_GURANTOR', 'LOAN_DURATION', 'STATUS', 'ADDRESS_TYPE',
       'ADDRESS_SLNO', 'SUB_STATUS', 'AGE', 'MOB', 'Target', 'cluster_no']]

In [190]:
final_Data_cluster = pd.concat([Final_train,Final_test], axis=0)

In [191]:
final_Data_cluster.to_csv("D:\cust_coll_data_cluster.csv")

In [192]:
final_Data_cluster

,AGMTNO,CHANNEL,PAYMENT_TYPE,AREA_CODE,DEALER_TYPE,DEALER_CODE,MAINDEALER_CODE,EXECUTIVE_CODE,PRODUCT_CODE,MODEL_CODE,...,CIBIL_SCORE_GURANTOR,LOAN_DURATION,STATUS,ADDRESS_TYPE,ADDRESS_SLNO,SUB_STATUS,AGE,MOB,Target,cluster_no
0,TN3000TW0039449,0,0,3000,0,0,0,0,0,0,...,0.0,9,0,0,1,0,25,0,0,4
1,GJ3031TW10471,0,1,3031,0,1,1,1,0,1,...,0.0,11,0,0,1,0,37,0,0,4
2,GJ3031TW0012554,0,1,3031,0,2,2,2,0,1,...,0.0,11,0,0,1,0,38,0,0,4
3,GJ3031TW10521,0,1,3031,0,1,1,1,1,2,...,0.0,11,0,1,1,0,36,0,0,4
4,TN3000TW0039724,0,1,3000,0,0,0,0,1,3,...,0.0,5,0,0,1,0,22,0,0,0
5,TN3006TW0071627,0,2,3006,1,3,3,3,2,4,...,0.0,11,0,1,1,0,50,0,0,1
6,TN3000TW0039601,0,0,3000,0,0,0,4,0,0,...,0.0,11,0,0,1,0,26,0,0,3
7,KA3009TW0095215,0,2,3009,0,4,4,5,2,5,...,0.0,6,1,1,1,1,32,14,1,1
8,GJ3031TW10624,0,3,3031,1,5,1,1,0,1,...,0.0,11,2,1,1,0,48,0,0,4
9,GJ3031TW10538,0,1,3031,0,1,1,6,1,6,...,0.0,11,0,0,1,0,32,0,0,3
